## Exampville Mode Choice Logsums

In [2]:
import os
import numpy as np
import pandas as pd
import larch
import larch.exampville
from larch import P, X

In [4]:
hh = pd.read_csv( larch.exampville.files.hh )
pp = pd.read_csv( larch.exampville.files.person )
tour = pd.read_csv( larch.exampville.files.tour )
skims = larch.OMX( larch.exampville.files.skims, mode='r' )

In [5]:
m = larch.read_metadata('./exampville_mode_choice.html')

In [6]:
raw = tour.merge(hh, on='HHID').merge(pp, on=('HHID', 'PERSONID'))
raw["HOMETAZi"] = raw["HOMETAZ"] - 1
raw["DTAZi"] = raw["DTAZ"] - 1
raw = raw[raw.TOURPURP == 1]
raw = raw.join(
    skims.get_rc_dataframe(
        raw.HOMETAZi, raw.DTAZi,
    )
)

### # Define model

In [8]:
# Define numbers as names for modes
DA,SR,Walk,Bike,Transit = 1,2,3,4,5
dfs = larch.DataFrames(co=raw, 
                       alt_codes=[DA,SR,Walk,Bike,Transit],
                       alt_names=['DA','SR','Walk','Bike','Transit'],
                       ch_name='TOUR_MODE')
# Set data
m.dataservice = dfs

In [9]:
logsums = pd.DataFrame(data=0.0, index=raw.index, columns=skims.TAZ_ID)

In [11]:
%%time
for destination_i, dtaz in enumerate(logsums.columns):
    # Get the LOS data for this destination
    new_data = skims.get_rc_dataframe(raw.HOMETAZi, destination_i)
    # Write data into model's dataservice
    dfs.data_co[new_data.columns] = new_data
    # Preprocessing 
    m.load_data()
    # Compute logsums
    logsums[dtaz] = m.logsums()

In [12]:
logsums

TAZ_ID,1,2,3,4,5,6,7,8,9,10,...,31,32,33,34,35,36,37,38,39,40
0,-2.935611,-2.660436,-1.745654,-2.229281,-2.403117,-1.624193,-2.689783,-2.704533,-3.258630,-2.718634,...,-1.702280,-1.717749,-2.340898,-2.760539,-1.029270,-2.249755,-2.048357,-2.160251,-2.286809,-1.959727
1,-2.935611,-2.660436,-1.745654,-2.229281,-2.403117,-1.624193,-2.689783,-2.704533,-3.258630,-2.718634,...,-1.702280,-1.717749,-2.340898,-2.760539,-1.029270,-2.249755,-2.048357,-2.160251,-2.286809,-1.959727
3,-2.935611,-2.660436,-1.745654,-2.229281,-2.403117,-1.624193,-2.689783,-2.704533,-3.258630,-2.718634,...,-1.702280,-1.717749,-2.340898,-2.760539,-1.029270,-2.249755,-2.048357,-2.160251,-2.286809,-1.959727
7,-3.046071,-2.769949,-1.823372,-2.320338,-2.494063,-1.705313,-2.785841,-2.806107,-3.365500,-2.830964,...,-1.784186,-1.797482,-2.430807,-2.863272,-1.100473,-2.343703,-2.133276,-2.247024,-2.380627,-2.047688
10,-3.051611,-2.775413,-1.827178,-2.324782,-2.498425,-1.709122,-2.790498,-2.811107,-3.370804,-2.836603,...,-1.788032,-1.801228,-2.435109,-2.868261,-1.103813,-2.348381,-2.137300,-2.251151,-2.385124,-2.051987
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20728,-1.077996,-0.700216,-3.010829,-1.287954,-0.675443,-0.529269,-1.322370,-0.830805,-1.836621,-0.783314,...,0.528568,-0.778369,0.479229,0.486074,-2.297550,-0.383914,-0.730914,-1.165841,-0.439654,-0.918778
20729,-1.076000,-0.697942,-3.006081,-1.285674,-0.673034,-0.503678,-1.320438,-0.828959,-1.834276,-0.781373,...,0.552873,-0.776292,0.495548,0.509936,-2.294579,-0.354081,-0.729007,-1.163952,-0.421577,-0.912944
20731,-1.076000,-0.697942,-3.006081,-1.285674,-0.673034,-0.503678,-1.320438,-0.828959,-1.834276,-0.781373,...,0.552873,-0.776292,0.495548,0.509936,-2.294579,-0.354081,-0.729007,-1.163952,-0.421577,-0.912944
20734,-1.068268,-0.689182,-2.987863,-1.276848,-0.663743,-0.410560,-1.312960,-0.821811,-1.825216,-0.773860,...,0.641614,-0.768272,0.555741,0.597167,-2.283161,-0.246771,-0.721653,-1.156644,-0.354410,-0.890508


In [13]:
logsums.to_pickle('./logsums.pkl.gz')